In [18]:
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [3]:
train_data

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [4]:
test_targets

array([ 7.2, 18.8, 19. , 27. , 22.2, 24.5, 31.2, 22.9, 20.5, 23.2, 18.6,
       14.5, 17.8, 50. , 20.8, 24.3, 24.2, 19.8, 19.1, 22.7, 12. , 10.2,
       20. , 18.5, 20.9, 23. , 27.5, 30.1,  9.5, 22. , 21.2, 14.1, 33.1,
       23.4, 20.1,  7.4, 15.4, 23.8, 20.1, 24.5, 33. , 28.4, 14.1, 46.7,
       32.5, 29.6, 28.4, 19.8, 20.2, 25. , 35.4, 20.3,  9.7, 14.5, 34.9,
       26.6,  7.2, 50. , 32.4, 21.6, 29.8, 13.1, 27.5, 21.2, 23.1, 21.9,
       13. , 23.2,  8.1,  5.6, 21.7, 29.6, 19.6,  7. , 26.4, 18.9, 20.9,
       28.1, 35.4, 10.2, 24.3, 43.1, 17.6, 15.4, 16.2, 27.1, 21.4, 21.5,
       22.4, 25. , 16.6, 18.6, 22. , 42.8, 35.1, 21.5, 36. , 21.9, 24.1,
       50. , 26.7, 25. ])

In [7]:
train_data.shape, train_targets.shape

((404, 13), (404,))

In [8]:
train_data[9]

array([  1.62864,   0.     ,  21.89   ,   0.     ,   0.624  ,   5.019  ,
       100.     ,   1.4394 ,   4.     , 437.     ,  21.2    , 396.9    ,
        34.41   ])

In [9]:
"""Computing the mean and the standard deviation of the training data inorder to normalize the data"""
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

In [22]:
train_data -= mean
train_data /= std
test_data -= mean
test_data /= std

In [25]:
def build_model():
  model = Sequential([
      Dense(64, activation = "swish"),
      Dense(64, activation = "swish"),
      Dense(1)
  ])
  model.compile(optimizer="rmsprop",loss="mse", metrics = ["mae"])
  return model

In [26]:
"""K-fold validation"""
k = 4
num_val_samples = len(train_data)//k
num_epochs = 500
all_scores = []
for i in range(k):
  val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
  partial_train_data = np.concatenate(
      [train_data[:i * num_val_samples],
       train_data[(i + 1) * num_val_samples:]],
       axis = 0)
  partial_train_targets = np.concatenate(
      [train_targets[:i * num_val_samples],
       train_targets[(i + 1) * num_val_samples:]],
       axis = 0)
  S_Net = build_model()
  S_Net.fit(partial_train_data, partial_train_targets,
            epochs=num_epochs,batch_size=16, verbose=0)
  val_mse, val_mae = S_Net.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)
  

In [27]:
all_scores

[2.1996326446533203, 2.2493484020233154, 2.5358028411865234, 2.240116596221924]

In [28]:
np.mean(all_scores)

2.3062251210212708